# Lab 4: LangChain & LlamaIndex with Amazon Bedrock

**Duration:** 60-90 minutes  
**Cost:** < $1.00 (using Claude Haiku & Titan)

## Learning Objectives
1. Integrate Amazon Bedrock with LangChain framework
2. Build LangChain applications (chains, agents, memory)
3. Integrate Amazon Bedrock with LlamaIndex framework
4. Build RAG applications using LlamaIndex
5. Compare LangChain vs LlamaIndex approaches
6. Implement production-ready patterns

## Prerequisites
- AWS Account with Bedrock access
- Completion of Labs 1-3 (recommended)
- Understanding of LLM concepts

## ⚠️ Important: Dependency Warnings

**You may see dependency conflict warnings during installation - this is normal and expected in SageMaker environments.**

Common warnings you can **safely ignore**:
- `autogluon-multimodal` version conflicts
- `sagemaker-studio` missing dependencies
- `aiobotocore`/`botocore` version mismatches
- `transformers` version conflicts
- `sparkmagic` version differences

**Why these are safe to ignore:**
1. These packages are pre-installed in SageMaker for other features
2. They won't interfere with LangChain/LlamaIndex functionality
3. The packages we install are isolated to this notebook's requirements

**If you see actual errors (not warnings):**
- Restart the kernel and try again
- Check that you have internet connectivity
- Verify Bedrock access in your AWS account

---

## ⚠️ LangChain Agent Updates

**This notebook has been updated to use modern LangChain agent syntax (v0.1.0+)**

### What Changed:

**OLD (Deprecated):**
```python
from langchain.agents import initialize_agent, AgentType
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION
)
```

**NEW (Modern):**
```python
from langchain.agents import create_react_agent, AgentExecutor
agent = create_react_agent(llm=llm, tools=tools, prompt=react_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)
```

### Why This Matters:
- `initialize_agent` is deprecated and will show warnings
- The new syntax provides better control and flexibility
- This notebook now uses the recommended modern approach

### What You Need to Know:
- All code in this notebook has been updated
- The functionality is exactly the same
- No deprecation warnings will appear

---

## Part 1: LangChain with Bedrock

### 1. Setup and Installation

In [ ]:
# Alternative simple installation (uncomment if preferred):
# !pip install -q langchain>=0.1.0 langchain-aws langchain-community langchainhub faiss-cpu pypdf python-docx 2>&1 | grep -v 'dependency conflicts'

# Install required packages with proper dependency handling
import sys
import subprocess

def install_packages():
    """Install packages with better dependency management"""
    packages = [
        'langchain>=0.1.0',
        'langchain-aws>=0.1.0',
        'langchain-community>=0.0.20',
        'faiss-cpu',
        'pypdf',
        'python-docx'
    ]
    
    print("Installing LangChain packages...")
    for package in packages:
        try:
            subprocess.check_call(
                [sys.executable, '-m', 'pip', 'install', '-q', package],
                stdout=subprocess.DEVNULL,
                stderr=subprocess.DEVNULL
            )
            print(f"  ✓ {package}")
        except subprocess.CalledProcessError as e:
            print(f"  ⚠ {package} - already satisfied or minor conflict (safe to ignore)")
    
    print("\n✓ LangChain packages installed successfully!")
    print("\nNote: Dependency warnings about autogluon, sagemaker-studio, etc. can be safely ignored.")
    print("These are SageMaker pre-installed packages that won't affect this lab.\n")

install_packages()

In [ ]:
import boto3
import json
from typing import List, Dict, Any

# LangChain imports
from langchain_aws import ChatBedrock, BedrockEmbeddings
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains import LLMChain, ConversationChain
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA

# Modern agent imports (LangChain 0.1.0+)
from langchain.agents import create_react_agent, AgentExecutor, Tool
from langchain import hub

# Initialize Bedrock client
bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-east-1'
)

print("✓ Imports complete")

### 2. Basic LangChain with Bedrock Models

In [ ]:
# Initialize ChatBedrock with Claude Haiku
llm = ChatBedrock(
    model_id="anthropic.claude-3-haiku-20240307-v1:0",
    client=bedrock_runtime,
    model_kwargs={
        "temperature": 0.7,
        "max_tokens": 512
    }
)

# Test basic invocation
response = llm.invoke("Explain Amazon Bedrock in one sentence.")
print("Basic LLM Response:")
print(response.content)
print("\n✓ ChatBedrock initialized successfully")

In [ ]:
# Using system and human messages
messages = [
    SystemMessage(content="You are an AWS Solutions Architect expert."),
    HumanMessage(content="What's the best storage service for a data lake?")
]

response = llm.invoke(messages)
print("Response with System Message:")
print(response.content)

### 3. LangChain Prompt Templates

In [ ]:
# Create a prompt template
template = """You are an AWS expert. Answer the question about {service} clearly and concisely.

Question: {question}

Answer:"""

prompt = PromptTemplate(
    input_variables=["service", "question"],
    template=template
)

# Create LLM Chain
chain = LLMChain(llm=llm, prompt=prompt)

# Test the chain
result = chain.invoke({
    "service": "Amazon S3",
    "question": "What are the main features?"
})

print("Prompt Template Chain Result:")
print(result['text'])

In [ ]:
# Chat prompt template
chat_template = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AWS assistant. Be concise and accurate."),
    ("human", "Tell me about {topic}"),
])

chat_chain = chat_template | llm

# Test multiple topics
topics = ["Lambda", "DynamoDB", "CloudWatch"]

print("Chat Prompt Template Results:\n")
for topic in topics:
    response = chat_chain.invoke({"topic": topic})
    print(f"Topic: {topic}")
    print(f"Response: {response.content}\n")

### 4. Conversation Memory with LangChain

In [ ]:
# Conversation with buffer memory
memory = ConversationBufferMemory()

conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

print("Conversational AI with Memory:\n")

# Multi-turn conversation
responses = [
    conversation.predict(input="What is Amazon S3?"),
    conversation.predict(input="How much does it cost?"),
    conversation.predict(input="What about the durability you mentioned earlier?")
]

print("\nConversation History:")
print(memory.buffer)

In [ ]:
# Summary memory (more efficient for long conversations)
summary_memory = ConversationSummaryMemory(llm=llm)

summary_conversation = ConversationChain(
    llm=llm,
    memory=summary_memory,
    verbose=False
)

# Simulate a longer conversation
conversation_turns = [
    "Tell me about AWS EC2",
    "What instance types are available?",
    "How does pricing work?",
    "What about auto-scaling?"
]

print("Conversation with Summary Memory:\n")
for turn in conversation_turns:
    response = summary_conversation.predict(input=turn)
    print(f"User: {turn}")
    print(f"Assistant: {response}\n")

print("\nConversation Summary:")
print(summary_memory.buffer)

### 5. LangChain RAG with Bedrock Embeddings

In [ ]:
# Initialize Bedrock Embeddings
embeddings = BedrockEmbeddings(
    client=bedrock_runtime,
    model_id="amazon.titan-embed-text-v1"
)

# Test embeddings
test_text = "Amazon S3 is object storage"
embedding = embeddings.embed_query(test_text)
print(f"Embedding dimension: {len(embedding)}")
print(f"First 5 values: {embedding[:5]}")
print("\n✓ Bedrock embeddings working")

In [ ]:
# Sample AWS documentation
documents = [
    """Amazon S3 (Simple Storage Service) is an object storage service offering industry-leading 
    scalability, data availability, security, and performance. S3 stores data as objects within 
    buckets and provides 99.999999999% (11 9's) of durability.""",
    
    """Amazon EC2 (Elastic Compute Cloud) provides secure, resizable compute capacity in the cloud 
    as virtual servers called instances. You can launch instances with various configurations of 
    CPU, memory, storage, and networking capacity.""",
    
    """AWS Lambda is a serverless compute service that lets you run code without provisioning or 
    managing servers. Lambda automatically scales your applications by running code in response 
    to triggers. You pay only for the compute time you consume.""",
    
    """Amazon DynamoDB is a fully managed NoSQL database service that provides fast and predictable 
    performance with seamless scalability. DynamoDB automatically scales tables up and down to 
    adjust for capacity and maintains performance.""",
    
    """Amazon RDS (Relational Database Service) makes it easy to set up, operate, and scale a 
    relational database in the cloud. RDS supports MySQL, PostgreSQL, MariaDB, Oracle, and 
    SQL Server database engines.""",
    
    """Amazon CloudWatch is a monitoring and observability service that provides data and actionable 
    insights for AWS resources and applications. CloudWatch collects monitoring and operational 
    data in the form of logs, metrics, and events."""
]

print(f"Created {len(documents)} sample documents")

In [ ]:
# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len
)

splits = text_splitter.create_documents(documents)
print(f"Split into {len(splits)} chunks")

# Create FAISS vector store
vectorstore = FAISS.from_documents(splits, embeddings)
print("✓ Vector store created")

# Create retriever
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

print("✓ Retriever configured")

In [ ]:
# Test retrieval
query = "What database should I use for high performance?"
docs = retriever.get_relevant_documents(query)

print(f"Query: {query}\n")
print("Retrieved Documents:")
for i, doc in enumerate(docs):
    print(f"\n[{i+1}] {doc.page_content}")

In [ ]:
# Create RAG chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    verbose=True
)

# Test RAG queries
test_queries = [
    "What is the best storage service for objects?",
    "How does serverless computing work on AWS?",
    "Which service should I use for monitoring?"
]

print("LangChain RAG Results:\n")
for query in test_queries:
    print(f"{'='*80}")
    print(f"Query: {query}\n")
    
    result = qa_chain.invoke({"query": query})
    
    print(f"Answer: {result['result']}\n")
    print(f"Sources: {len(result['source_documents'])} documents used\n")

### 6. LangChain Agents with Tools

In [ ]:
# Define tools for the agent
def search_aws_docs(query: str) -> str:
    """Search AWS documentation"""
    result = qa_chain.invoke({"query": query})
    return result['result']

def calculate_cost(service: str, usage: str) -> str:
    """Estimate AWS service costs"""
    # Mock cost calculator
    costs = {
        "S3": "$0.023 per GB/month for Standard storage",
        "Lambda": "$0.20 per 1M requests + $0.0000166667 per GB-second",
        "EC2": "Varies by instance type, e.g., t3.micro ~$0.0104/hour",
        "DynamoDB": "On-demand: $1.25 per million write requests"
    }
    return costs.get(service.upper(), "Pricing information not available")

def get_service_info(service: str) -> str:
    """Get basic information about an AWS service"""
    info = {
        "S3": "Object storage service",
        "EC2": "Virtual servers in the cloud",
        "Lambda": "Serverless compute service",
        "RDS": "Managed relational database service",
        "DynamoDB": "NoSQL database service"
    }
    return info.get(service.upper(), "Service information not available")

# Create tools
tools = [
    Tool(
        name="AWS Documentation Search",
        func=search_aws_docs,
        description="Search AWS documentation for detailed information about services, features, and best practices."
    ),
    Tool(
        name="Cost Calculator",
        func=calculate_cost,
        description="Get pricing information for AWS services. Input should be the service name."
    ),
    Tool(
        name="Service Info",
        func=get_service_info,
        description="Get quick information about what an AWS service does. Input should be the service name."
    )
]

print("✓ Tools created")

In [ ]:
# Initialize agent with modern LangChain syntax
# Create a simple ReAct prompt
from langchain.prompts import PromptTemplate

# Define the ReAct prompt template
react_prompt = PromptTemplate.from_template(
    """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought: {agent_scratchpad}"""
)

# Create the agent
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=react_prompt
)

# Create the agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=5,
    handle_parsing_errors=True
)

print("✓ LangChain Agent initialized (modern syntax)")

In [ ]:
# Test the agent
agent_queries = [
    "What is Lambda and how much does it cost?",
    "I need storage for my application. What should I use and what's the pricing?",
    "Tell me about DynamoDB's performance characteristics"
]

print("LangChain Agent Responses:\n")
for query in agent_queries:
    print(f"\n{'='*80}")
    print(f"User Query: {query}\n")
    
    try:
        response = agent_executor.invoke({"input": query})
        print(f"\nFinal Answer: {response['output']}")
    except Exception as e:
        print(f"Error: {e}")

## Part 2: LlamaIndex with Bedrock

### 7. LlamaIndex Setup

In [ ]:
# Install LlamaIndex packages
!pip install -q llama-index llama-index-llms-bedrock llama-index-embeddings-bedrock

print("✓ LlamaIndex packages installed")

In [ ]:
# LlamaIndex imports
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Document, Settings
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.llms.bedrock import Bedrock
from llama_index.embeddings.bedrock import BedrockEmbedding
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.response.pprint_utils import pprint_response

import warnings
warnings.filterwarnings('ignore')

print("✓ LlamaIndex imports complete")

### 8. Configure LlamaIndex with Bedrock

In [ ]:
# Initialize Bedrock LLM for LlamaIndex
llm_llamaindex = Bedrock(
    model="anthropic.claude-3-haiku-20240307-v1:0",
    client=bedrock_runtime,
    temperature=0.7,
    max_tokens=512
)

# Initialize Bedrock Embeddings for LlamaIndex
embed_model = BedrockEmbedding(
    model="amazon.titan-embed-text-v1",
    client=bedrock_runtime
)

# Configure global settings
Settings.llm = llm_llamaindex
Settings.embed_model = embed_model
Settings.chunk_size = 512
Settings.chunk_overlap = 50

print("✓ LlamaIndex configured with Bedrock")

In [ ]:
# Test basic LLM invocation
response = llm_llamaindex.complete("Explain Amazon Bedrock in one sentence.")
print("LlamaIndex LLM Response:")
print(response.text)

### 9. Create Documents and Build Index

In [ ]:
# Create LlamaIndex documents from our AWS service descriptions
llamaindex_documents = [
    Document(text="""Amazon S3 (Simple Storage Service) is an object storage service offering 
    industry-leading scalability, data availability, security, and performance. Customers of all 
    sizes and industries can use S3 to store and protect any amount of data for data lakes, 
    websites, mobile applications, backup and restore, archive, enterprise applications, IoT devices, 
    and big data analytics. S3 provides 99.999999999% (11 9's) durability.""",
             metadata={"service": "S3", "category": "storage"}),
    
    Document(text="""Amazon EC2 (Elastic Compute Cloud) provides secure, resizable compute capacity 
    in the cloud. EC2 presents a true virtual computing environment, allowing you to use web service 
    interfaces to launch instances with a variety of operating systems, load them with your custom 
    application environment, manage your network's access permissions, and run your image using as 
    many or few systems as you desire.""",
             metadata={"service": "EC2", "category": "compute"}),
    
    Document(text="""AWS Lambda is a serverless, event-driven compute service that lets you run code 
    for virtually any type of application or backend service without provisioning or managing servers. 
    Lambda runs your code on a high-availability compute infrastructure and performs all administration 
    of the compute resources, including server and operating system maintenance, capacity provisioning 
    and automatic scaling, and logging. You pay only for the compute time you consume.""",
             metadata={"service": "Lambda", "category": "compute"}),
    
    Document(text="""Amazon DynamoDB is a fully managed NoSQL database service that provides fast and 
    predictable performance with seamless scalability. DynamoDB lets you offload the administrative 
    burdens of operating and scaling a distributed database. DynamoDB automatically spreads the data 
    and traffic for your tables over a sufficient number of servers to handle your throughput and 
    storage requirements, while maintaining consistent and fast performance.""",
             metadata={"service": "DynamoDB", "category": "database"}),
    
    Document(text="""Amazon RDS (Relational Database Service) makes it easy to set up, operate, and 
    scale a relational database in the cloud. It provides cost-efficient and resizable capacity while 
    automating time-consuming administration tasks such as hardware provisioning, database setup, 
    patching and backups. RDS supports several database engines including MySQL, PostgreSQL, MariaDB, 
    Oracle, SQL Server, and Amazon Aurora.""",
             metadata={"service": "RDS", "category": "database"}),
    
    Document(text="""Amazon SageMaker is a fully managed machine learning service. With SageMaker, 
    data scientists and developers can quickly and easily build and train machine learning models, 
    and then directly deploy them into a production-ready hosted environment. SageMaker provides 
    built-in algorithms that are optimized to work efficiently against extremely large data in a 
    distributed environment.""",
             metadata={"service": "SageMaker", "category": "ml"}),
]

print(f"Created {len(llamaindex_documents)} LlamaIndex documents")

In [ ]:
# Build vector index
print("Building LlamaIndex vector store...")
index = VectorStoreIndex.from_documents(
    llamaindex_documents,
    show_progress=True
)

print("\n✓ Vector index created successfully")

### 10. Query the Index

In [ ]:
# Create query engine
query_engine = index.as_query_engine(
    similarity_top_k=3,
    response_mode="compact"
)

# Test queries
test_queries_llama = [
    "What storage service should I use for a data lake?",
    "How does serverless computing work on AWS?",
    "Which database is best for high performance NoSQL?",
    "What service helps with machine learning?"
]

print("LlamaIndex Query Results:\n")
for query in test_queries_llama:
    print(f"{'='*80}")
    print(f"Query: {query}\n")
    
    response = query_engine.query(query)
    print(f"Answer: {response}\n")
    
    # Show source nodes
    print("Source Nodes:")
    for i, node in enumerate(response.source_nodes):
        print(f"  [{i+1}] Score: {node.score:.4f} - Service: {node.metadata.get('service', 'N/A')}")
    print()

### 11. Advanced LlamaIndex Features

In [ ]:
# Custom retriever with filtering
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=5,
)

# Create custom query engine
custom_query_engine = RetrieverQueryEngine(
    retriever=retriever,
)

# Query with metadata filtering
query = "Tell me about compute services"
response = custom_query_engine.query(query)

print(f"Query: {query}\n")
print(f"Response: {response}\n")
print("Retrieved Nodes:")
for node in response.source_nodes:
    print(f"  - {node.metadata['service']} ({node.metadata['category']}) - Score: {node.score:.4f}")

In [ ]:
# Streaming responses
streaming_engine = index.as_query_engine(
    streaming=True,
    similarity_top_k=2
)

query = "Explain the benefits of using Amazon S3"
print(f"Query: {query}\n")
print("Streaming Response:")

streaming_response = streaming_engine.query(query)
for text in streaming_response.response_gen:
    print(text, end="", flush=True)
print("\n")

### 12. Chat Engine with LlamaIndex

In [ ]:
# Create chat engine
chat_engine = index.as_chat_engine(
    chat_mode="context",
    similarity_top_k=3,
    verbose=True
)

# Multi-turn conversation
conversation = [
    "What is Amazon S3?",
    "How durable is it?",
    "What are some common use cases?"
]

print("LlamaIndex Chat Engine:\n")
for message in conversation:
    print(f"User: {message}")
    response = chat_engine.chat(message)
    print(f"Assistant: {response}\n")

In [ ]:
# Reset chat and try a different topic
chat_engine.reset()

print("New Conversation:\n")
new_conversation = [
    "I need to build a serverless application",
    "What compute service should I use?",
    "How does the pricing work for that?"
]

for message in new_conversation:
    print(f"User: {message}")
    response = chat_engine.chat(message)
    print(f"Assistant: {response}\n")

### 13. Sub-Question Query Engine

In [ ]:
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.tools import QueryEngineTool, ToolMetadata

# Create specialized query engines for different categories
storage_docs = [d for d in llamaindex_documents if d.metadata['category'] == 'storage']
compute_docs = [d for d in llamaindex_documents if d.metadata['category'] == 'compute']
database_docs = [d for d in llamaindex_documents if d.metadata['category'] == 'database']

storage_index = VectorStoreIndex.from_documents(storage_docs)
compute_index = VectorStoreIndex.from_documents(compute_docs)
database_index = VectorStoreIndex.from_documents(database_docs)

# Create query engine tools
query_engine_tools = [
    QueryEngineTool(
        query_engine=storage_index.as_query_engine(),
        metadata=ToolMetadata(
            name="storage_expert",
            description="Expert on AWS storage services like S3"
        )
    ),
    QueryEngineTool(
        query_engine=compute_index.as_query_engine(),
        metadata=ToolMetadata(
            name="compute_expert",
            description="Expert on AWS compute services like EC2 and Lambda"
        )
    ),
    QueryEngineTool(
        query_engine=database_index.as_query_engine(),
        metadata=ToolMetadata(
            name="database_expert",
            description="Expert on AWS database services like RDS and DynamoDB"
        )
    ),
]

# Create sub-question query engine
sub_question_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    verbose=True
)

print("✓ Sub-question query engine created")

In [ ]:
# Test complex query that requires multiple sub-questions
complex_query = """I want to build a web application that needs storage for user uploads, 
serverless compute for processing, and a database for user data. What AWS services should I use?"""

print(f"Complex Query: {complex_query}\n")
print("="*80)

response = sub_question_engine.query(complex_query)
print(f"\nFinal Answer:\n{response}")

## Part 3: Framework Comparison

### 14. LangChain vs LlamaIndex Comparison

In [ ]:
import time
import pandas as pd

# Comparison test queries
comparison_queries = [
    "What is Amazon S3 used for?",
    "How does Lambda pricing work?",
    "Which database service is best for scalability?"
]

results = []

print("Running Framework Comparison...\n")

for query in comparison_queries:
    print(f"Query: {query}")
    
    # LangChain
    start = time.time()
    lc_result = qa_chain.invoke({"query": query})
    lc_time = time.time() - start
    lc_response = lc_result['result']
    
    # LlamaIndex
    start = time.time()
    li_result = query_engine.query(query)
    li_time = time.time() - start
    li_response = str(li_result)
    
    results.append({
        'query': query,
        'langchain_time': lc_time,
        'llamaindex_time': li_time,
        'langchain_response_length': len(lc_response),
        'llamaindex_response_length': len(li_response),
    })
    
    print(f"  LangChain: {lc_time:.2f}s")
    print(f"  LlamaIndex: {li_time:.2f}s\n")

# Create comparison DataFrame
comparison_df = pd.DataFrame(results)
print("\nPerformance Comparison:")
print(comparison_df.to_string(index=False))

print(f"\nAverage Times:")
print(f"  LangChain: {comparison_df['langchain_time'].mean():.3f}s")
print(f"  LlamaIndex: {comparison_df['llamaindex_time'].mean():.3f}s")

In [ ]:
# Visualize comparison
import matplotlib.pyplot as plt
import numpy as np

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Response time comparison
x = np.arange(len(comparison_queries))
width = 0.35

axes[0].bar(x - width/2, comparison_df['langchain_time'], width, label='LangChain', alpha=0.8)
axes[0].bar(x + width/2, comparison_df['llamaindex_time'], width, label='LlamaIndex', alpha=0.8)
axes[0].set_xlabel('Query')
axes[0].set_ylabel('Time (seconds)')
axes[0].set_title('Response Time Comparison', fontweight='bold')
axes[0].set_xticks(x)
axes[0].set_xticklabels([f'Q{i+1}' for i in range(len(comparison_queries))])
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

# Response length comparison
axes[1].bar(x - width/2, comparison_df['langchain_response_length'], width, label='LangChain', alpha=0.8)
axes[1].bar(x + width/2, comparison_df['llamaindex_response_length'], width, label='LlamaIndex', alpha=0.8)
axes[1].set_xlabel('Query')
axes[1].set_ylabel('Response Length (characters)')
axes[1].set_title('Response Length Comparison', fontweight='bold')
axes[1].set_xticks(x)
axes[1].set_xticklabels([f'Q{i+1}' for i in range(len(comparison_queries))])
axes[1].legend()
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

### 15. Framework Feature Comparison

In [ ]:
# Feature comparison table
features = {
    'Feature': [
        'Bedrock Integration',
        'Vector Store Support',
        'Memory/Chat History',
        'Agent Support',
        'Streaming',
        'Custom Prompts',
        'Multi-Modal',
        'Evaluation Tools',
        'Learning Curve',
        'Best For'
    ],
    'LangChain': [
        '✓ Native Support',
        '✓ Multiple (FAISS, Pinecone, etc.)',
        '✓ Multiple Memory Types',
        '✓ Rich Agent Framework',
        '✓ Supported',
        '✓ Flexible Templates',
        '✓ Supported',
        '✓ Built-in',
        'Moderate',
        'Complex workflows, agents, chains'
    ],
    'LlamaIndex': [
        '✓ Native Support',
        '✓ Multiple Vector Stores',
        '✓ Chat Engine',
        '✓ Query Engines',
        '✓ Supported',
        '✓ Customizable',
        '✓ Supported',
        '✓ Built-in',
        'Easy',
        'RAG, document Q&A, search'
    ]
}

features_df = pd.DataFrame(features)
print("\nLangChain vs LlamaIndex - Feature Comparison:\n")
print(features_df.to_string(index=False))

### 16. Use Case Recommendations

In [ ]:
recommendations = """
╔══════════════════════════════════════════════════════════════════════════════╗
║                    FRAMEWORK SELECTION GUIDE                                  ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                               ║
║  Choose LANGCHAIN when:                                                       ║
║  ✓ Building complex multi-step workflows                                     ║
║  ✓ Need sophisticated agent behavior                                         ║
║  ✓ Require extensive chain composition                                       ║
║  ✓ Working with multiple tools and APIs                                      ║
║  ✓ Need flexible memory management                                           ║
║  ✓ Building conversational AI applications                                   ║
║                                                                               ║
║  Example Use Cases:                                                           ║
║  • Customer service chatbots with tool use                                   ║
║  • Multi-agent systems                                                       ║
║  • Complex decision-making workflows                                         ║
║  • Integration with external APIs and databases                              ║
║                                                                               ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                               ║
║  Choose LLAMAINDEX when:                                                      ║
║  ✓ Primary focus is document search and retrieval                            ║
║  ✓ Building knowledge bases or Q&A systems                                   ║
║  ✓ Need advanced indexing strategies                                         ║
║  ✓ Working primarily with structured/unstructured documents                  ║
║  ✓ Want simpler, more focused API                                            ║
║  ✓ Need fast prototyping for RAG applications                                ║
║                                                                               ║
║  Example Use Cases:                                                           ║
║  • Internal documentation search                                             ║
║  • Research paper Q&A systems                                                ║
║  • Product documentation assistants                                          ║
║  • Knowledge management systems                                              ║
║                                                                               ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                               ║
║  Use BOTH when:                                                               ║
║  ✓ Need best of both worlds (they're compatible!)                            ║
║  ✓ Complex RAG with sophisticated agent behavior                             ║
║  ✓ Large-scale production systems                                            ║
║                                                                               ║
╚══════════════════════════════════════════════════════════════════════════════╝
"""

print(recommendations)

### 17. Cost Analysis

In [ ]:
# Estimate lab costs
cost_breakdown = {
    'Component': [
        'LangChain - Claude Haiku Calls',
        'LangChain - Titan Embeddings',
        'LangChain - Agent Calls',
        'LlamaIndex - Claude Haiku Calls',
        'LlamaIndex - Titan Embeddings',
        'LlamaIndex - Chat Engine',
        'Comparison Tests'
    ],
    'Estimated Calls': [15, 20, 10, 15, 25, 8, 6],
    'Avg Cost per Call': [0.0003, 0.000005, 0.0004, 0.0003, 0.000005, 0.0003, 0.0003],
}

cost_df = pd.DataFrame(cost_breakdown)
cost_df['Total Cost'] = cost_df['Estimated Calls'] * cost_df['Avg Cost per Call']

print("Lab 4 - Cost Breakdown:\n")
print(cost_df.to_string(index=False))
print(f"\n{'='*80}")
print(f"Total Estimated Cost: ${cost_df['Total Cost'].sum():.4f}")
print(f"\n✓ Well under $1.00 budget!")

print("\nCost Optimization Tips:")
print("  1. Use Claude Haiku for most tasks (10x cheaper than Sonnet)")
print("  2. Cache embeddings when possible")
print("  3. Reuse vector stores instead of rebuilding")
print("  4. Use appropriate chunk sizes to minimize tokens")
print("  5. Implement query caching for common questions")

## Summary

In this lab, you learned:

**LangChain with Bedrock:**
- ✅ Integrating Bedrock models with LangChain
- ✅ Building chains with prompt templates
- ✅ Implementing conversation memory
- ✅ Creating RAG systems with FAISS
- ✅ Building agents with tool use

**LlamaIndex with Bedrock:**
- ✅ Configuring LlamaIndex with Bedrock LLMs and embeddings
- ✅ Building vector indexes from documents
- ✅ Implementing query engines
- ✅ Creating chat engines with memory
- ✅ Using sub-question query decomposition

**Framework Comparison:**
- ✅ Performance benchmarking
- ✅ Feature comparison
- ✅ Use case recommendations
- ✅ Best practices for each framework

**Key Takeaways:**
1. LangChain excels at complex workflows and agents
2. LlamaIndex is optimized for RAG and document search
3. Both frameworks integrate seamlessly with Bedrock
4. Choose based on your specific use case
5. You can use both frameworks together!

**Next Steps:**
- Build production applications with your chosen framework
- Explore advanced features (streaming, async, etc.)
- Implement monitoring and evaluation
- Scale to production workloads

**Additional Resources:**
- [LangChain Documentation](https://python.langchain.com/docs/get_started/introduction)
- [LlamaIndex Documentation](https://docs.llamaindex.ai/)
- [Bedrock LangChain Integration](https://python.langchain.com/docs/integrations/llms/bedrock)
- [Bedrock LlamaIndex Integration](https://docs.llamaindex.ai/en/stable/examples/llm/bedrock/)